<a href="https://colab.research.google.com/github/JafarunnishaShaik/Fraud-Transaction-Detection/blob/main/Copy_of_Analyzing_Amazon_Sales_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

# This will open a prompt for you to upload the file
files.upload()

Saving zomato-schema.xlsx to zomato-schema.xlsx


{'zomato-schema.xlsx': b'PK\x03\x04\x14\x00\x08\x08\x08\x00\x1d_cZ\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x18\x00\x00\x00xl/drawings/drawing1.xml\x9d\xd0]n\xc20\x0c\x07\xf0\x13\xec\x0eU\xdeiZ\x18\x13C\x14^\xd0N0\x0e\xe0%n\x1b\x91\x8f\xca\x0e\xa3\xdc~\xd1J6i{\x01\x1em\xcb?\xf9\xef\xcdnt\xb6\xf8Db\x13|#\xea\xb2\x12\x05z\x15\xb4\xf1]#\x0e\xefo\xb3\x95(8\x82\xd7`\x83\xc7F\\\x90\xc5n\xfb\xb4\x195\xad\xcf\xbc\xa7"\xed{^\xa7\xb2\x11}\x8c\xc3ZJV=:\xe02\x0c\xe8\xd3\xb4\r\xe4 \xa6\x92:\xa9\t\xceIvV\xce\xab\xeaE\xf2@\x08\x9a{\xc4\xb8\x9f&\xe2\xea\xc1\x03\x9a\x03\xe3\xf3\xfeM\xd7\x84\xb65\n\xf7A\x9d\x1c\xfa8!\x84\x16b\xfa\x05\xf7f\xe0\xac\xa9\x07\xaeQ=P\xfc\x01\xc6\x7f\x823\x8a\x02\x876\x96*\xb8\xeb)\xd9HB\xfd<\t8\xfe\x1a\xf5\xdd\xc8R\xbe\xca\xd5_\xc8\xdd\x14\xc7\x01\x1dO\xc3,\xb9Cz\xc8\x87\xb1&^\xbe\x93eFw\xee\x81\xb7h\x03\x1d\x81\xcb\xc8\xb88\xf8\xe3\xdd\xb1*\xc96\xb5(+l\xb1^\xde\xad\xcc\xb3"\xb7_PK\x07\x08\x07bi\x83\x05\x01\x00\x00\x07\x03\x00\x00PK\x03\x04\x14\x00\x08\x08\x08\x00\x1d_

In [2]:
import os

# List all the files in the environment
print(os.listdir())

['.config', 'zomato-schema.xlsx', 'sample_data']


In [3]:
import pandas as pd

def load_data(file_path):
    """Load data from an Excel file and display available sheets."""
    # Load the Excel file
    xls = pd.ExcelFile(file_path)

    # Display available sheets
    print("Available sheets:", xls.sheet_names)

    # Read all sheets into a dictionary of DataFrames
    return {sheet: pd.read_excel(file_path, sheet) for sheet in xls.sheet_names}

# Use the correct filename you uploaded
file_path = 'zomato-schema.xlsx'

# Call the function to load the dataset
dataframes = load_data(file_path)

Available sheets: ['users', 'restaurants', 'food', 'menu', 'orders', 'delivery_partner', 'order_details']


In [4]:
#Example: Display the first 5 rows of the 'orders' sheet
dataframes['orders'].head()

,order_id,user_id,r_id,amount,date,partner_id,delivery_time,delivery_rating,restaurant_rating
0,1001,1,1,550,2022-05-10,1,25,5,3.0
1,1002,1,2,415,2022-05-26,1,19,5,2.0
2,1003,1,3,240,2022-06-15,5,29,4,NaN
3,1004,1,3,240,2022-06-29,4,42,3,5.0
4,1005,1,3,220,2022-07-10,1,58,1,4.0


In [5]:
import pandas as pd

def clean_data(df):
    """Clean data by handling missing values, duplicates, and incorrect data types."""

    # 1. Remove duplicate rows
    print(f"Duplicates before removal: {df.duplicated().sum()}")
    df.drop_duplicates(inplace=True)

    # 2. Handle missing values (fill with 0 for numeric columns)
    numeric_cols = df.select_dtypes(include='number').columns
    df[numeric_cols] = df[numeric_cols].fillna(0)

    # 3. Convert date columns to datetime format
    for col in df.columns:
        if 'date' in col.lower() or 'time' in col.lower():
            df[col] = pd.to_datetime(df[col], errors='coerce')

    # Drop rows with invalid dates
    df.dropna(subset=[col for col in df.columns if pd.api.types.is_datetime64_any_dtype(df[col])], inplace=True)

    print("✅ Data cleaned successfully!")
    return df

# Apply cleaning to all sheets
cleaned_data = {sheet: clean_data(df) for sheet, df in dataframes.items()}

# Verify cleaned sheets
for sheet, df in cleaned_data.items():
    print(f"\n✅ {sheet} - Cleaned Shape: {df.shape}")

Duplicates before removal: 0
✅ Data cleaned successfully!
Duplicates before removal: 0
✅ Data cleaned successfully!
Duplicates before removal: 0
✅ Data cleaned successfully!
Duplicates before removal: 0
✅ Data cleaned successfully!
Duplicates before removal: 0
✅ Data cleaned successfully!
Duplicates before removal: 0
✅ Data cleaned successfully!
Duplicates before removal: 0
✅ Data cleaned successfully!

✅ users - Cleaned Shape: (7, 4)

✅ restaurants - Cleaned Shape: (5, 3)

✅ food - Cleaned Shape: (11, 3)

✅ menu - Cleaned Shape: (15, 4)

✅ orders - Cleaned Shape: (25, 9)

✅ delivery_partner - Cleaned Shape: (5, 2)

✅ order_details - Cleaned Shape: (50, 3)


In [6]:
# Save all cleaned sheets to CSV files
for sheet, df in cleaned_data.items():
    file_name = f"cleaned_{sheet}.csv"
    df.to_csv(file_name, index=False)
    print(f"✅ Saved: {file_name}")

✅ Saved: cleaned_users.csv
✅ Saved: cleaned_restaurants.csv
✅ Saved: cleaned_food.csv
✅ Saved: cleaned_menu.csv
✅ Saved: cleaned_orders.csv
✅ Saved: cleaned_delivery_partner.csv
✅ Saved: cleaned_order_details.csv


In [7]:
cleaned_data['orders'].head()

,order_id,user_id,r_id,amount,date,partner_id,delivery_time,delivery_rating,restaurant_rating
0,1001,1,1,550,2022-05-10,1,1970-01-01 00:00:00.000000025,5,3.0
1,1002,1,2,415,2022-05-26,1,1970-01-01 00:00:00.000000019,5,2.0
2,1003,1,3,240,2022-06-15,5,1970-01-01 00:00:00.000000029,4,0.0
3,1004,1,3,240,2022-06-29,4,1970-01-01 00:00:00.000000042,3,5.0
4,1005,1,3,220,2022-07-10,1,1970-01-01 00:00:00.000000058,1,4.0


In [8]:
import pandas as pd

# Load the orders DataFrame
orders_df = cleaned_data['orders']

# Ensure 'date' is in datetime format
orders_df['date'] = pd.to_datetime(orders_df['date'], errors='coerce')

# Check for missing dates (if any)
print("Missing Dates: ", orders_df['date'].isnull().sum())

# Display the entire DataFrame to confirm
print(orders_df.head(25))

Missing Dates:  0
    order_id  user_id  r_id  amount       date  partner_id  \
0       1001        1     1     550 2022-05-10           1   
1       1002        1     2     415 2022-05-26           1   
2       1003        1     3     240 2022-06-15           5   
3       1004        1     3     240 2022-06-29           4   
4       1005        1     3     220 2022-07-10           1   
5       1006        2     1     950 2022-06-10           2   
6       1007        2     2     530 2022-06-23           3   
7       1008        2     3     240 2022-07-07           5   
8       1009        2     4     300 2022-07-17           4   
9       1010        2     5     650 2022-07-31           1   
10      1011        3     1     450 2022-05-10           2   
11      1012        3     4     180 2022-05-20           5   
12      1013        3     2     230 2022-05-30           4   
13      1014        3     2     230 2022-06-11           2   
14      1015        3     2     230 2022-06-22      

In [9]:
# Ensure 'order_date' is in datetime format
orders_df = cleaned_data['orders']

# Extract Year, Month, and Year-Month
orders_df['year'] = orders_df['date'].dt.year
orders_df['month'] = orders_df['date'].dt.month
orders_df['year_month'] = orders_df['date'].dt.to_period('M')

# Verify the new columns
orders_df.head(25)

,order_id,user_id,r_id,amount,date,partner_id,delivery_time,delivery_rating,restaurant_rating,year,month,year_month
0,1001,1,1,550,2022-05-10,1,1970-01-01 00:00:00.000000025,5,3.0,2022,5,2022-05
1,1002,1,2,415,2022-05-26,1,1970-01-01 00:00:00.000000019,5,2.0,2022,5,2022-05
2,1003,1,3,240,2022-06-15,5,1970-01-01 00:00:00.000000029,4,0.0,2022,6,2022-06
3,1004,1,3,240,2022-06-29,4,1970-01-01 00:00:00.000000042,3,5.0,2022,6,2022-06
4,1005,1,3,220,2022-07-10,1,1970-01-01 00:00:00.000000058,1,4.0,2022,7,2022-07
5,1006,2,1,950,2022-06-10,2,1970-01-01 00:00:00.000000016,5,0.0,2022,6,2022-06
6,1007,2,2,530,2022-06-23,3,1970-01-01 00:00:00.000000060,1,5.0,2022,6,2022-06
7,1008,2,3,240,2022-07-07,5,1970-01-01 00:00:00.000000033,4,5.0,2022,7,2022-07
8,1009,2,4,300,2022-07-17,4,1970-01-01 00:00:00.000000041,1,0.0,2022,7,2022-07
9,1010,2,5,650,2022-07-31,1,1970-01-01 00:00:00.000000067,1,4.0,2022,7,2022-07


In [10]:
# Yearly Sales Trend
yearly_sales = orders_df.groupby('year')['amount'].sum().reset_index()
print("Yearly Sales Trend:\n", yearly_sales)

Yearly Sales Trend:
    year  amount
0  2022   10490


In [11]:
# Monthly Sales Trend (Across All Years)
monthly_sales = orders_df.groupby('month')['amount'].sum().reset_index()
print("Monthly Sales Trend:\n", monthly_sales)

Monthly Sales Trend:
    month  amount
0      5    2425
1      6    3220
2      7    4845


In [12]:
# Year-Month Sales Trend
year_month_sales = orders_df.groupby('year_month')['amount'].sum().reset_index()
print("Year-Month Sales Trend:\n", year_month_sales)

Year-Month Sales Trend:
   year_month  amount
0    2022-05    2425
1    2022-06    3220
2    2022-07    4845


In [13]:
# Save to CSV files for Power BI
yearly_sales.to_csv('yearly_sales.csv', index=False)
monthly_sales.to_csv('monthly_sales.csv', index=False)
year_month_sales.to_csv('year_month_sales.csv', index=False)

print("✅ Sales trend data saved successfully!")

✅ Sales trend data saved successfully!


In [14]:
# Save to CSV files for Power BI
yearly_sales.to_csv('yearly_sales.csv', index=False)
monthly_sales.to_csv('monthly_sales.csv', index=False)
year_month_sales.to_csv('year_month_sales.csv', index=False)

print("✅ Sales trend data saved successfully!")

✅ Sales trend data saved successfully!


In [15]:
# Total Sales (Sum of the 'total_amount' column)(Key Metric)
#Total sales
total_sales = orders_df['amount'].sum()
print(f"💰 Total Sales: {total_sales}")

💰 Total Sales: 10490


In [17]:
import pandas as pd

# Load the cleaned data again
file_path = "/content/zomato-schema.xlsx"  # Update if needed
cleaned_data = pd.read_excel(file_path, sheet_name=None)

# Extract relevant DataFrames
order_details_df = cleaned_data['order_details']

# Now, rerun the code
item_quantity = order_details_df.groupby('f_id')['order_id'].count().reset_index()
item_quantity.columns = ['item_id', 'quantity']

# Display the result
print(item_quantity)

    item_id  quantity
0         1         5
1         2         1
2         3        13
3         4         8
4         5         3
5         6         4
6         7         1
7         8         4
8         9         3
9        10         4
10       11         4


In [18]:
# Calculate the quantity sold for each food item (f_id)
#Top selling items
item_quantity = order_details_df.groupby('f_id')['order_id'].count().reset_index()

# Rename columns for clarity
item_quantity.columns = ['food_item_id', 'quantity']

# Display the result
print(item_quantity.head())

   food_item_id  quantity
0             1         5
1             2         1
2             3        13
3             4         8
4             5         3


In [19]:
# Merge with the food DataFrame for details
food_df = cleaned_data['food']

# Combine quantity with food details
item_sales_summary = pd.merge(item_quantity, food_df, left_on='food_item_id', right_on='f_id', how='left')

# Drop the duplicate id column
item_sales_summary.drop('f_id', axis=1, inplace=True)

# Display the enriched summary
print(item_sales_summary.head())

   food_item_id  quantity           f_name     type
0             1         5    Non-veg Pizza  Non-veg
1             2         1        Veg Pizza      Veg
2             3        13  Choco Lava cake      Veg
3             4         8    Chicken Wings  Non-veg
4             5         3  Chicken Popcorn  Non-veg


In [20]:
# Save the item sales summary to a CSV file
item_sales_summary.to_csv('item_sales_summary.csv', index=False)
print("✅ Item sales summary saved as 'item_sales_summary.csv'")

✅ Item sales summary saved as 'item_sales_summary.csv'


In [21]:
yearly_sales.to_csv('yearly_sales.csv', index=False)
monthly_sales.to_csv('monthly_sales.csv', index=False)
year_month_sales.to_csv('year_month_sales.csv', index=False)

In [22]:
# Check the delivery_time range
print(orders_df['delivery_time'].describe())

count                               25
mean     1970-01-01 00:00:00.000000039
min      1970-01-01 00:00:00.000000016
25%      1970-01-01 00:00:00.000000026
50%      1970-01-01 00:00:00.000000038
75%      1970-01-01 00:00:00.000000050
max      1970-01-01 00:00:00.000000070
Name: delivery_time, dtype: object


In [23]:
# Extract nanoseconds from delivery_time and convert to minutes
orders_df['delivery_time_minutes'] = orders_df['delivery_time'].dt.nanosecond / (60 * 1e9)

# Check the result
print(orders_df[['delivery_time', 'delivery_time_minutes']].head())

                  delivery_time  delivery_time_minutes
0 1970-01-01 00:00:00.000000025           4.166667e-10
1 1970-01-01 00:00:00.000000019           3.166667e-10
2 1970-01-01 00:00:00.000000029           4.833333e-10
3 1970-01-01 00:00:00.000000042           7.000000e-10
4 1970-01-01 00:00:00.000000058           9.666667e-10


In [24]:
# Convert nanoseconds to minutes
#Adjusting Nanoseconds to Minutes
#minutes=Nanoseconds/60*10power 9
orders_df['delivery_time_minutes'] = orders_df['delivery_time'].dt.nanosecond / (60 * 1e9)

# Scale up for better visibility
orders_df['delivery_time_minutes'] *= 1e9

# Display the updated delivery time in minutes
print(orders_df[['delivery_time', 'delivery_time_minutes']].head())

                  delivery_time  delivery_time_minutes
0 1970-01-01 00:00:00.000000025               0.416667
1 1970-01-01 00:00:00.000000019               0.316667
2 1970-01-01 00:00:00.000000029               0.483333
3 1970-01-01 00:00:00.000000042               0.700000
4 1970-01-01 00:00:00.000000058               0.966667


In [25]:
# Calculate average delivery time by partner
avg_delivery_time = orders_df.groupby('partner_id')['delivery_time_minutes'].mean().reset_index()
avg_delivery_time.columns = ['partner_id', 'avg_delivery_time']

print(avg_delivery_time.head())

   partner_id  avg_delivery_time
0           1           0.769048
1           2           0.661111
2           3           0.575000
3           4           0.691667
4           5           0.487500


In [26]:
# Average delivery rating by partner
avg_delivery_rating = orders_df.groupby('partner_id')['delivery_rating'].mean().reset_index()
avg_delivery_rating.columns = ['partner_id', 'avg_delivery_rating']

print(avg_delivery_rating.head())

   partner_id  avg_delivery_rating
0           1             2.857143
1           2             3.000000
2           3             4.000000
3           4             3.000000
4           5             3.500000


In [27]:
# Identify fastest and slowest delivery partners
fastest_partner = avg_delivery_time.nsmallest(1, 'avg_delivery_time')
slowest_partner = avg_delivery_time.nlargest(1, 'avg_delivery_time')

print("🚀 Fastest Delivery Partner:\n", fastest_partner)
print("🐢 Slowest Delivery Partner:\n", slowest_partner)

🚀 Fastest Delivery Partner:
    partner_id  avg_delivery_time
4           5             0.4875
🐢 Slowest Delivery Partner:
    partner_id  avg_delivery_time
0           1           0.769048


In [28]:
# Average delivery rating by partner
avg_delivery_rating = orders_df.groupby('partner_id')['delivery_rating'].mean().reset_index()
avg_delivery_rating.columns = ['partner_id', 'avg_delivery_rating']

print("📊 Average Delivery Rating by Partner:\n", avg_delivery_rating)

📊 Average Delivery Rating by Partner:
    partner_id  avg_delivery_rating
0           1             2.857143
1           2             3.000000
2           3             4.000000
3           4             3.000000
4           5             3.500000


In [29]:
# Merge delivery time and rating
delivery_performance = pd.merge(avg_delivery_time, avg_delivery_rating, on='partner_id')

# Save to CSV for Power BI
delivery_performance.to_csv('delivery_performance.csv', index=False)

print("✅ Delivery performance metrics saved as 'delivery_performance.csv'")
print(delivery_performance.head())

✅ Delivery performance metrics saved as 'delivery_performance.csv'
   partner_id  avg_delivery_time  avg_delivery_rating
0           1           0.769048             2.857143
1           2           0.661111             3.000000
2           3           0.575000             4.000000
3           4           0.691667             3.000000
4           5           0.487500             3.500000


In [30]:
# Best performer (highest rating, fastest time)
best_performer = delivery_performance.nsmallest(1, ['avg_delivery_time']).nlargest(1, ['avg_delivery_rating'])

# Worst performer (slowest time, lowest rating)
worst_performer = delivery_performance.nlargest(1, ['avg_delivery_time']).nsmallest(1, ['avg_delivery_rating'])

print("🏆 Best Overall Performer:\n", best_performer)
print("💢 Worst Overall Performer:\n", worst_performer)

🏆 Best Overall Performer:
    partner_id  avg_delivery_time  avg_delivery_rating
4           5             0.4875                  3.5
💢 Worst Overall Performer:
    partner_id  avg_delivery_time  avg_delivery_rating
0           1           0.769048             2.857143


In [31]:
# Step 1: Identify Repeat Customers
# Count how many orders each customer placed
customer_orders = orders_df.groupby('user_id')['order_id'].count().reset_index()

# Rename columns for clarity
customer_orders.columns = ['user_id', 'order_count']

# Identify repeat customers (more than one order)
repeat_customers = customer_orders[customer_orders['order_count'] > 1]

print("🔁 Repeat Customers:")
print(repeat_customers)

# Save repeat customers to CSV
repeat_customers.to_csv('repeat_customers.csv', index=False)
print("✅ Repeat customers saved as 'repeat_customers.csv'")

🔁 Repeat Customers:
   user_id  order_count
0        1            5
1        2            5
2        3            5
3        4            5
4        5            5
✅ Repeat customers saved as 'repeat_customers.csv'


In [32]:
# Step 2: Calculate Average Order Value (AOV)
# Total sales value and total orders
total_sales_value = orders_df['amount'].sum()
total_orders_count = orders_df['order_id'].nunique()

# Calculate AOV
average_order_value = total_sales_value / total_orders_count

print(f"📊 Average Order Value (AOV): {average_order_value:.2f}")

📊 Average Order Value (AOV): 419.60


In [33]:
# Analyze Restaurant Performance
# Ensure you have 'orders_df' DataFrame with relevant columns
# Restaurant Performance Metrics
restaurant_performance = orders_df.groupby('r_id').agg(
    total_orders=('order_id', 'count'),
    total_revenue=('amount', 'sum'),
    avg_restaurant_rating=('restaurant_rating', 'mean')
).reset_index()

print("📊 Restaurant Performance Metrics:")
print(restaurant_performance)

# Save restaurant performance to CSV
restaurant_performance.to_csv('restaurant_performance.csv', index=False)

# Top-Performing Restaurant (Highest Revenue and Best Rating)
top_restaurant = restaurant_performance.nlargest(1, ['total_revenue', 'avg_restaurant_rating'])
print("🏆 Top-Performing Restaurant:")
print(top_restaurant)

# Save top-performing restaurant to CSV
top_restaurant.to_csv('top_restaurant.csv', index=False)

# Lowest-Performing Restaurant (Lowest Revenue and Lowest Rating)
lowest_restaurant = restaurant_performance.nsmallest(1, ['total_revenue', 'avg_restaurant_rating'])
print("💢 Lowest-Performing Restaurant:")
print(lowest_restaurant)

# Save lowest-performing restaurant to CSV
lowest_restaurant.to_csv('lowest_restaurant.csv', index=False)

print("✅ Restaurant performance analysis completed and saved!")

📊 Restaurant Performance Metrics:
   r_id  total_orders  total_revenue  avg_restaurant_rating
0     1             5           3050               1.000000
1     2             8           3570               1.375000
2     3             4            940               3.500000
3     4             5           1480               2.200000
4     5             3           1450               3.666667
🏆 Top-Performing Restaurant:
   r_id  total_orders  total_revenue  avg_restaurant_rating
1     2             8           3570                  1.375
💢 Lowest-Performing Restaurant:
   r_id  total_orders  total_revenue  avg_restaurant_rating
2     3             4            940                    3.5
✅ Restaurant performance analysis completed and saved!


In [34]:
# Total revenue
total_revenue = orders_df['amount'].sum()
print(f"💰 Total Revenue: {total_revenue}")

# Average revenue per order
avg_revenue_per_order = orders_df['amount'].mean()
print(f"📊 Average Revenue per Order: {avg_revenue_per_order:.2f}")

# Revenue by restaurant (r_id)
revenue_by_restaurant = orders_df.groupby('r_id')['amount'].sum().reset_index()
revenue_by_restaurant.columns = ['r_id', 'total_revenue']
print("🍽️ Revenue by Restaurant:\n", revenue_by_restaurant.head())

# Revenue by partner (partner_id)
revenue_by_partner = orders_df.groupby('partner_id')['amount'].sum().reset_index()
revenue_by_partner.columns = ['partner_id', 'total_revenue']
print("🤝 Revenue by Delivery Partner:\n", revenue_by_partner.head())

# Save revenue insights for Power BI
revenue_by_restaurant.to_csv('revenue_by_restaurant.csv', index=False)
revenue_by_partner.to_csv('revenue_by_partner.csv', index=False)

print("✅ Revenue insights saved successfully!")

💰 Total Revenue: 10490
📊 Average Revenue per Order: 419.60
🍽️ Revenue by Restaurant:
    r_id  total_revenue
0     1           3050
1     2           3570
2     3            940
3     4           1480
4     5           1450
🤝 Revenue by Delivery Partner:
    partner_id  total_revenue
0           1           3085
1           2           3320
2           3           1460
3           4           1415
4           5           1210
✅ Revenue insights saved successfully!


In [35]:
# Merge cuisine data with orders
orders_with_cuisine = pd.merge(orders_df, cleaned_data['restaurants'], on='r_id')

# Total Orders by Cuisine
cuisine_orders = orders_with_cuisine.groupby('cuisine')['order_id'].count().reset_index()
cuisine_orders.columns = ['cuisine', 'total_orders']
print("🍴 Total Orders by Cuisine:\n", cuisine_orders)

# Total Revenue by Cuisine
cuisine_revenue = orders_with_cuisine.groupby('cuisine')['amount'].sum().reset_index()
cuisine_revenue.columns = ['cuisine', 'total_revenue']
print("💰 Total Revenue by Cuisine:\n", cuisine_revenue)

# Average Order Value (AOV) by Cuisine
cuisine_aov = orders_with_cuisine.groupby('cuisine')['amount'].mean().reset_index()
cuisine_aov.columns = ['cuisine', 'avg_order_value']
print("📊 AOV by Cuisine:\n", cuisine_aov)

# Top-Performing Cuisine (by revenue and orders)
top_cuisine_orders = cuisine_orders.nlargest(1, 'total_orders')
top_cuisine_revenue = cuisine_revenue.nlargest(1, 'total_revenue')
print("🏆 Most Ordered Cuisine:\n", top_cuisine_orders)
print("💵 Top Revenue-Generating Cuisine:\n", top_cuisine_revenue)

# Customer Preferences (Repeat Customers by Cuisine)
repeat_orders = orders_with_cuisine.groupby(['user_id', 'cuisine']).size().reset_index(name='order_count')
most_preferred_cuisine = repeat_orders.groupby('cuisine')['order_count'].sum().reset_index()
most_preferred_cuisine.columns = ['cuisine', 'repeat_orders']
print("🔁 Repeat Orders by Cuisine:\n", most_preferred_cuisine)

# Save Insights to CSV for Power BI
cuisine_insights = pd.merge(cuisine_orders, cuisine_revenue, on='cuisine')
cuisine_insights = pd.merge(cuisine_insights, cuisine_aov, on='cuisine')
cuisine_insights.to_csv('cuisine_insights.csv', index=False)

print("✅ Geographical (Cuisine) Insights saved successfully!")

🍴 Total Orders by Cuisine:
         cuisine  total_orders
0      American             8
1       Chinese             3
2       Italian             5
3  North Indian             4
4  South Indian             5
💰 Total Revenue by Cuisine:
         cuisine  total_revenue
0      American           3570
1       Chinese           1450
2       Italian           3050
3  North Indian            940
4  South Indian           1480
📊 AOV by Cuisine:
         cuisine  avg_order_value
0      American       446.250000
1       Chinese       483.333333
2       Italian       610.000000
3  North Indian       235.000000
4  South Indian       296.000000
🏆 Most Ordered Cuisine:
     cuisine  total_orders
0  American             8
💵 Top Revenue-Generating Cuisine:
     cuisine  total_revenue
0  American           3570
🔁 Repeat Orders by Cuisine:
         cuisine  repeat_orders
0      American              8
1       Chinese              3
2       Italian              5
3  North Indian              4
4  South I